In [4]:
from dataclasses import dataclass
from kv import KV
from sqlmodel import create_engine, SQLModel, delete, Field, Session, select

In [2]:
engine = create_engine("sqlite:///db.sqlite", echo=True)

class User(SQLModel, table=True):
  id: int | None = Field(default=None, primary_key=True)
  name: str
  age: int

SQLModel.metadata.create_all(engine)

2024-07-06 16:29:31,848 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-06 16:29:31,850 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2024-07-06 16:29:31,851 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-06 16:29:31,853 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user")
2024-07-06 16:29:31,855 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-06 16:29:31,858 INFO sqlalchemy.engine.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	age INTEGER NOT NULL, 
	PRIMARY KEY (id)
)


2024-07-06 16:29:31,860 INFO sqlalchemy.engine.Engine [no key 0.00113s] ()
2024-07-06 16:29:31,872 INFO sqlalchemy.engine.Engine COMMIT


In [3]:
users = [
  User(name="Alice", age=24),
  User(name="Bob", age=27),
  User(name="Charlie", age=33),
]

with Session(engine) as session:
  session.add_all(users)
  session.commit()

2024-07-06 16:29:34,491 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-06 16:29:34,496 INFO sqlalchemy.engine.Engine INSERT INTO user (name, age) VALUES (?, ?) RETURNING id
2024-07-06 16:29:34,498 INFO sqlalchemy.engine.Engine [generated in 0.00024s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('Alice', 24)
2024-07-06 16:29:34,500 INFO sqlalchemy.engine.Engine INSERT INTO user (name, age) VALUES (?, ?) RETURNING id
2024-07-06 16:29:34,501 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('Bob', 27)
2024-07-06 16:29:34,503 INFO sqlalchemy.engine.Engine INSERT INTO user (name, age) VALUES (?, ?) RETURNING id
2024-07-06 16:29:34,504 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('Charlie', 33)
2024-07-06 16:29:34,507 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
with Session(engine) as session:
  session.execute(delete(User)) 
  session.commit()

2024-07-06 16:31:28,064 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-06 16:31:28,067 INFO sqlalchemy.engine.Engine DELETE FROM user
2024-07-06 16:31:28,068 INFO sqlalchemy.engine.Engine [generated in 0.00141s] ()
2024-07-06 16:31:28,072 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
@dataclass
class User:
  uuid: str
  username: str
  age: int

users = [
  User(uuid='1', username='alice', age=30),
  User(uuid='2', username='bob', age=25),
  User(uuid='3', username='charlie', age=35),
]

In [5]:
kv = KV.of('sql+sqlite:///kv.sqlite;Table=kv', User)

In [6]:
for user in users:
  (await kv.insert(user.uuid, user)).unsafe()